In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


def segment_lesion_auto(image_rgb, min_lesion_frac=0.005):
    gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)

    inv = 255 - gray
    _, th = cv2.threshold(inv, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    th = cv2.medianBlur(th, 5)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel, iterations=1)
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN,  kernel, iterations=1)

    num, labels = cv2.connectedComponents(th)
    if num > 1:
        areas = [np.sum(labels == i) for i in range(1, num)]
        max_lbl = 1 + int(np.argmax(areas))
        mask = (labels == max_lbl)
    else:
        mask = th.astype(bool)

    if mask.mean() < min_lesion_frac:
        mask[:] = False
    return mask

def mask_by_region_auto(image_rgb, lesion_ratio=0.6, bg_ratio=0.4, mask_value=(0,0,0), seed=None):
    """
    自動找病灶，再將病灶區遮蔽 lesion_ratio、非病灶區遮蔽 bg_ratio。
    回傳 (masked_img, lesion_mask_bool, masked_pixels_uint8)
    """
    assert image_rgb.ndim == 3 and image_rgb.shape[2] == 3, "image_rgb must be HxWx3 RGB."
    H, W, _ = image_rgb.shape
    lesion_mask = segment_lesion_auto(image_rgb)  # Bool mask

    lesion_idx = np.flatnonzero(lesion_mask.ravel())
    bg_idx = np.flatnonzero(~lesion_mask.ravel())

    rng = np.random.default_rng(seed)
    n_lesion = int(min(len(lesion_idx), max(0, round(lesion_ratio * len(lesion_idx)))))
    n_bg     = int(min(len(bg_idx),     max(0, round(bg_ratio     * len(bg_idx)))))

    pick_lesion = rng.choice(lesion_idx, size=n_lesion, replace=False) if n_lesion > 0 else np.array([], dtype=int)
    pick_bg     = rng.choice(bg_idx,     size=n_bg,     replace=False) if n_bg > 0     else np.array([], dtype=int)
    pick_all    = np.concatenate([pick_lesion, pick_bg])

    masked_pixels = np.zeros((H * W,), dtype=np.uint8)
    if pick_all.size > 0:
        masked_pixels[pick_all] = 255
    masked_pixels = masked_pixels.reshape(H, W)

    masked_img = image_rgb.copy()
    if isinstance(mask_value, (tuple, list)) and len(mask_value) == 3:
        masked_img.reshape(-1, 3)[pick_all] = mask_value
    else:
        masked_img.reshape(-1, 3)[pick_all] = (mask_value, mask_value, mask_value)

    return masked_img, lesion_mask, masked_pixels


def show_triplet(image_path):
    bgr = cv2.imread(image_path)
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

    masked_img, lesion_mask, masked_pixels = mask_by_region_auto(
        rgb, lesion_ratio=0.8, bg_ratio=0.1
    )

    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.imshow(rgb); plt.title("Original"); plt.axis("off")
    plt.subplot(1,3,2); plt.imshow(lesion_mask, cmap="gray"); plt.title("Lesion Mask"); plt.axis("off")
    plt.subplot(1,3,3); plt.imshow(masked_img); plt.title("Masked (L80% / BG10%)"); plt.axis("off")
    plt.tight_layout(); plt.savefig("./tt.png", dpi=800, bbox_inches="tight"); plt.show()

show_triplet("./t.jpg")


In [ ]:
img = cv2.imread("./t.jpg")
plt.figure(figsize=(12, 4))
ax = plt.subplot(131)
ax.axis("off")
plt.imshow(img[:, :, ::-1])

ax = plt.subplot(132)
ax.axis("off")
blurred = cv2.bilateralFilter(img, d=5, sigmaColor=25, sigmaSpace=25)
plt.imshow(blurred[:, :, ::-1])

ax = plt.subplot(133)
ax.axis("off")
blurred = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
plt.imshow(blurred[:, :, ::-1])

plt.savefig("blue_show.png", dpi=400, bbox_inches="tight")
plt.show()

In [ ]:
import os
import random
import cv2
import numpy as np
import albumentations as A
from tqdm import tqdm

dataset_path = "/home/hank52052/Dataset/isic/HAM10000/yolo_format/train/"
target_count = 6705  

def gamma_adjust_red_channel(image, **kwargs):
    image = image.astype(np.int32)
    image[:, :, 2] = np.clip(image[:, :, 2] + np.random.randint(-30, 30), 0, 255)
    return image.astype(np.uint8)

augmentation_methods = [
    A.HorizontalFlip(p=1.0),
    A.VerticalFlip(p=1.0),
    A.Rotate(limit=(-30, 30), p=1.0),
    A.Affine(scale=(1.0, 1.3), translate_percent=(0.0, 0.1), rotate=(-30, 30), shear=(-10, 10), p=1.0),
    A.Resize(height=300, width=300, p=1.0),  
    A.RandomCrop(height=256, width=256, p=1.0), 
    A.Perspective(scale=(0.05, 0.1), p=1.0),
    A.ElasticTransform(alpha=1.0, sigma=50, p=1.0),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=1.0),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=1.0),
    A.Lambda(image=gamma_adjust_red_channel, p=1.0),  
    A.MotionBlur(blur_limit=(3, 7), p=1.0),
]

class_counts = {
    "AKIEC": 327,
    "BCC": 514,
    "BKL": 1099,
    "DF": 115,
    "MEL": 1113,
    "NV": 6705,
    "VASC": 142
}

for class_name, count in tqdm(class_counts.items(), desc="Data Augmentation"):
    class_dir = os.path.join(dataset_path, class_name)
    images = [f for f in os.listdir(class_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

    if count >= target_count:
        continue

    while len(images) < target_count:
        img_name = random.choice(images) 
        img_path = os.path.join(class_dir, img_name)
        
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        num_transforms = random.randint(2, 4)
        selected_transforms = random.sample(augmentation_methods, num_transforms)
        augmentation = A.Compose(selected_transforms)

        augmented = augmentation(image=image)['image']

        new_img_name = f"aug_{len(images)}_{img_name}"
        new_img_path = os.path.join(class_dir, new_img_name)
        cv2.imwrite(new_img_path, cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))
        
        images.append(new_img_name)  # 更新列表

print("✅ 影像增強完成，每個類別都擴增到 6705 張！")


In [ ]:
import cv2
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
import random

import cv2
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
import random
import textwrap


def generate_diagonal_cross_kernel(size=13):
    kernel = np.zeros((size, size), dtype=np.uint8)
    center = size // 2

    for i in range(0, size, 2):
        kernel[center, i] = 1
        kernel[i, center] = 1

    for i in range(0, size, 2):
        kernel[i, i] = 1
        kernel[i, size - 1 - i] = 1
    
    kernel[center, center - 1:center + 2] = 0
    kernel[center - 1:center + 2, center] = 0
    kernel[center - 1, center - 1] = 0
    kernel[center - 1, center + 1] = 0
    kernel[center + 1, center - 1] = 0
    kernel[center + 1, center + 1] = 0

    return kernel

kernel_struct = generate_diagonal_cross_kernel(13)

img_path = "../../Dataset/isic/HAM10000/images/ISIC_0024748.jpg" 
skin_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
top_hat = cv2.morphologyEx(skin_image, cv2.MORPH_BLACKHAT, kernel_struct)
_, binary_mask = cv2.threshold(top_hat, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

kernel_open = cv2.getStructuringElement(cv2.MORPH_CROSS, (9,9))
binary_mask_cleaned = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel_open)

image = cv2.imread(img_path)

image = cv2.inpaint(image, binary_mask_cleaned, inpaintRadius=5, flags=cv2.INPAINT_TELEA)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

augmentation_methods = [
    A.HorizontalFlip(p=1.0),
    A.VerticalFlip(p=1.0),
    A.Rotate(limit=(-30, 30), p=1.0),
    A.Affine(scale=(1.0, 1.3), translate_percent=(0.0, 0.1), rotate=(-30, 30), shear=(-10, 10), p=1.0),
    A.Perspective(scale=(0.05, 0.1), p=1.0),
    A.ElasticTransform(alpha=1.0, sigma=50, p=1.0),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=1.0),
    A.MotionBlur(blur_limit=(3, 7), p=1.0),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=1.0)
]


augmented_images = [image] 
titles = ["Original"]

for _ in range(8):
    num_transforms = random.randint(2, 4) 
    selected_transforms = random.sample(augmentation_methods, num_transforms)
    augmentation = A.Compose(selected_transforms)

    augmented = augmentation(image=image)["image"]
    augmented_images.append(augmented)

    transform_names = [str(t).split("(")[0] for t in selected_transforms]
    formatted_title = "\n".join(textwrap.wrap(", ".join(transform_names), width=25))  
    titles.append(formatted_title)

fig, axes = plt.subplots(3, 3, figsize=(12, 12))

for ax, img, title in zip(axes.ravel(), augmented_images, titles):
    ax.imshow(img)
    ax.set_title(title, fontsize=12, pad=10)  
    ax.axis("off")

plt.tight_layout()
plt.savefig("./aug_show.png", dpi=400, bbox_inches="tight")
plt.show()
